In [75]:
import pandas as pd

In [76]:
from langchain_core.vectorstores import InMemoryVectorStore  

In [97]:
from langchain_openai import OpenAIEmbeddings

In [184]:
from langchain_text_splitters import RecursiveCharacterTextSplitter  

In [77]:
from openai import OpenAI 

In [78]:
import requests

In [79]:
import json

In [80]:
from dotenv import load_dotenv
import os

# Подключаем все переменные из окружения
load_dotenv()
# Подключаем ключ для LLM-модели
LLM_API_KEY = os.getenv("LLM_API_KEY")
# Подключаем ключ для EMBEDDER-модели
EMBEDDER_API_KEY = os.getenv("EMBEDDER_API_KEY")

In [98]:
BASE_URL = 'https://openrouter.ai/api/v1'

In [99]:
client = OpenAI(
    base_url=BASE_URL,
    api_key=LLM_API_KEY,
)

In [82]:
questions = pd.read_csv('questions.csv')

In [83]:
corpus = pd.read_csv('train_data.csv')

In [84]:
corpus.head()

,id,annotation,tags,text
0,doc_001,Светлана из Казани дает частные уроки английск...,"['Начать бизнес', 'Самозанятые', 'Свое дело', ...",## Кто такой самозанятый?\n\nПо закону самозан...
1,doc_002,"Елене назначили социальное пособие на ребенка,...","['Защитить права', 'Банки', 'Банковская карта'...",Первым делом нужно попросить банк проверить ма...
2,doc_003,Самый надежный способ не оказаться в долгах — ...,"['Кредиты', 'Долги', 'Просрочки', 'Ипотека', '...",## Не переоценивайте свои финансовые возможнос...
3,doc_004,"Друзья Александра то и дело хвастаются, что по...","['Инвестиции', 'Ценные бумаги', 'Фондовая бирж...",Просто прийти на биржу и купить ценные бумаги ...
4,doc_005,Вы взяли в микрофинансовой организации заем на...,"['Займы', 'Долги', 'Риски', 'Защитить права']","## МФО больше нет в госреестре. Значит, она за..."


In [85]:
corpus.iloc[0]['annotation']

'Светлана из Казани дает частные уроки английского языка. Она неплохо зарабатывает, но весь ее доход — неофициальный. Из-за этого ей сложно получить кредит и визу в другую страну. Но недавно Светлана узнала, что можно получить статус самозанятого, платить небольшой налог — и проблем со справками о доходе не будет.Рассказываем, кто может зарегистрироваться как самозанятый и для чего это делать.'

In [86]:
corpus.iloc[0]['tags'].strip('[]').split(', ')

["'Начать бизнес'", "'Самозанятые'", "'Свое дело'", "'Налоги'"]

In [87]:
def row_to_doc(row): 
    return ( 
    f"""<Annotation>
    {row['annotation']}
    </Annotation>
    <Tags>
    {row['tags']}
    </Tags>
    <Text>
    {row['text']}
    </Text>
    """)

In [88]:
corpus['doc'] = corpus.apply(row_to_doc, axis=1) 

In [89]:
docs = list(corpus['doc'])

In [101]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small", base_url=BASE_URL, api_key=LLM_API_KEY)

In [197]:
class Chunker:
    def __init__(self, corpus):
        self.corpus = corpus
        self.splitter = RecursiveCharacterTextSplitter(  
            chunk_size=500,      # Maximum chunk size  
            chunk_overlap=100,    # Overlap between chunks for context  
            separators=["\n\n", "\n", " ", ""] 
        ) 
        self.chunks = self.chunk_corpus()

    def chunk_corpus(self):
        result = []
        for _, row in self.corpus.iterrows():
            tags = row['tags']
            annotation = row['annotation']
            text = row['text']
            result = result + self.chunk_doc(annotation, tags, text)
        chunks = [] 
        for c in result:
            if len(c) > 1000:
                chunks = chunks + self.splitter.split_text(text)
            else:
                chunks = chunks + [c]
        return chunks

    def chunk_doc(self, annotation, tags, text):
        chunks = self.chunk_text(text)
        tagged_annotation = self.tag_text(annotation, 'Annotation')
        tagged_tags = self.tag_text(annotation, 'Tags')
        return ['\n'.join([tagged_annotation, tagged_tags, chunk]) for chunk in chunks]
        
    @staticmethod
    def tag_text(text, tag):        
        opening_tag = f'<{tag}>'
        closing_tag = f'</{tag}>'
        return f'{opening_tag}\n{text}\n{closing_tag}'

    @classmethod
    def tag_chunk_info(cls, chunk):
        header_closing = '</Header>'
        if header_closing not in chunk:
            return cls.tag_text(chunk, 'Information')
        header, info = chunk.split(header_closing)
        tagged_info = cls.tag_text(info, 'Information')
        return header_closing.join([header, tagged_info])
    
    @classmethod
    def chunk_text(cls, text):
        lines_to_replace = []
        for line in text.split('\n'):
            if '#' in line:
                lines_to_replace.append(line)
        text_ = text
        for line in lines_to_replace:
            tagged_header = cls.tag_text(line.replace('#', '').strip(), 'Header')
            tagged_header = '\n'.join(['<Delimiter>', tagged_header])
            text_ = text_.replace(
                line,
                tagged_header
            )
        result = [x.strip() for x in text_.split('<Delimiter>') if x.strip() not in ['\n', '', '\t']]
        result = [cls.tag_chunk_info(x) for x in result]
        return result

In [198]:
chunker = Chunker(corpus)

In [199]:
max(len(c) for c in chunker.chunks)

1000

In [200]:
vectorstore = InMemoryVectorStore.from_texts(  
    chunker.chunks,  
    embedding=embeddings, 
)

In [202]:
len(chunker.chunks)

40706

In [203]:
vectorstore.dump('my_vdb.db')

In [205]:
questions.head().iloc[1]['Вопрос']

'Как действовать вкладчику при отзыве лицензии, учитывая лимит безопасной суммы?'

In [206]:
results = vectorstore.similarity_search(query='Как действовать вкладчику при отзыве лицензии, учитывая лимит безопасной суммы?', k=10)

In [209]:
results[0].page_content

'<Annotation>\nВсе банковские вклады и счета частных лиц, малых и средних предприятий застрахованы. Если у банка отзывают лицензию, вкладчики гарантированно получают свои средства в пределах 1,4 миллиона рублей.\n</Annotation>\n<Tags>\nВсе банковские вклады и счета частных лиц, малых и средних предприятий застрахованы. Если у банка отзывают лицензию, вкладчики гарантированно получают свои средства в пределах 1,4 миллиона рублей.\n</Tags>\n<Header>\nСколько я получу, если открывал счета в разных отделениях одного банка?\n</Header><Information>\n\n\nОни считаются вкладом в одном банке. Все ваши активы (вклады и счета) АСВ просуммирует. Из расчета этой общей суммы вы и получите страховую выплату, но не больше 1,4 млн рублей.\n</Information>'

In [211]:
merged_text = '\n'.join([doc.page_content for doc in results])

In [224]:
def english_or_spanish(text: str) -> str:
    translation_model = 'mistralai/mistral-small-3.2-24b-instruct'
    completion = client.chat.completions.create(
        model=translation_model,
        messages=[
            {
                "role": "user",
                "content": f"Translate this text to english: {text}"
            }
        ]
    )
    return completion

In [225]:
res = english_or_spanish(merged_text)

In [231]:
knowledge = Chunker.tag_text(merged_text, 'Knowledge')

In [232]:
completion = client.chat.completions.create(
  extra_headers={
    "HTTP-Referer": "<YOUR_SITE_URL>", # Optional. Site URL for rankings on openrouter.ai.
    "X-Title": "<YOUR_SITE_NAME>", # Optional. Site title for rankings on openrouter.ai.
  },
  extra_body={},
  model="x-ai/grok-3-mini",
  messages=[
    {
      "role": "user",
      "content": f"Answer the question, using information given to you. Only use information that is relevant to the question. Answer in russian. <Question>Как действовать вкладчику при отзыве лицензии, учитывая лимит безопасной суммы?</Question>{knowledge}"
    }
  ]
)

In [233]:
print(completion.choices[0].message.content)

Если у вашего банка отозвана лицензия, ваши банковские вклады и счета застрахованы, и вы можете получить свои средства в пределах лимита безопасной суммы — 1,4 миллиона рублей. Вот как действовать на основе доступной информации:

1. **Проверьте тип вашего счета.** Возмещение зависит от того, какой у вас счет (например, обычный счет, счет для бизнеса или счет эскроу). Для обычного счета физического лица вы можете запросить перевод средств на счет в другом банке-участнике системы страхования вкладов.

2. **Подайте заявку на возмещение.** Обратитесь в Агентство по страхованию вкладов (АСВ) или через банк-агент. Если у вас есть карта «Мир», укажите её номер в заявлении — это упростит процесс. Вы также можете получить деньги наличными, если выберете этот вариант.

3. **Учтите лимит.** АСВ просуммирует все ваши вклады и счета в этом банке (даже если они в разных отделениях) и выплатит сумму в пределах 1,4 миллиона рублей. Если у вас был вклад в иностранной валюте, она будет пересчитана в руб

In [235]:
print(f"Answer the question, using information given to you. Only use information that is relevant to the question. Answer in russian. <Question>Как действовать вкладчику при отзыве лицензии, учитывая лимит безопасной суммы?</Question>{knowledge}")

Answer the question, using information given to you. Only use information that is relevant to the question. Answer in russian. <Question>Как действовать вкладчику при отзыве лицензии, учитывая лимит безопасной суммы?</Question><Knowledge>
<Annotation>
Все банковские вклады и счета частных лиц, малых и средних предприятий застрахованы. Если у банка отзывают лицензию, вкладчики гарантированно получают свои средства в пределах 1,4 миллиона рублей.
</Annotation>
<Tags>
Все банковские вклады и счета частных лиц, малых и средних предприятий застрахованы. Если у банка отзывают лицензию, вкладчики гарантированно получают свои средства в пределах 1,4 миллиона рублей.
</Tags>
<Header>
Сколько я получу, если открывал счета в разных отделениях одного банка?
</Header><Information>


Они считаются вкладом в одном банке. Все ваши активы (вклады и счета) АСВ просуммирует. Из расчета этой общей суммы вы и получите страховую выплату, но не больше 1,4 млн рублей.
</Information>
<Annotation>
Все банковски